In [ ]:
!pip install pytube
!pip install ffmpy3

     |████████████████████████████████| 56 kB 2.8 MB/s eta 0:00:011


In [51]:
from pytube import YouTube
import os
import numpy as np
import matplotlib.pyplot as plt
import ffmpy3
from pydub import AudioSegment
import librosa as lr
import soundfile as sf
from pydub.playback import play


In [18]:
WORKING_DIR = "."
RAW_DATA_DIR = "./raw_data/"
WAV_DATA_DIR = "./wav_data/"
WAV_DATA_RESAMPLED_DIR ="./wav_data_resampled/"

In [13]:
!mkdir raw_data
!mkdir wav_data
!mkdir wav_data_resampled

In [14]:
def get_mp4_from_url(link,folder):
    """writes a wav audio given a youtube link
    
    Input
    -------------------
    link : a youtube link string 
    folder : destination folder name
    
    Output
    -------------------
    result : boolean to check if successful 
    """
    result = False
    try:
        yt_object = YouTube(link)
        audio_file = yt_object.streams.filter(only_audio=True, file_extension='mp4').first()
        audio_file.download(folder)
        result = True
        
    except:
        print("something went wrong!!")
    finally:
        return result
        

Testing function

In [19]:
url = 'https://youtu.be/tC45XiXljUE'
get_mp4_from_url(url,RAW_DATA_DIR)

True

In [30]:
def mp4_to_wav(filename,src_folder,dst_folder):
    """
    converts mp4 file to wav file with the same name and places it in dst_folder
    
    Input
    --------------------
    filename : name of the file, must be a .mp4 
    dst_folder : name of the destination folder
    src_folder : name of the folder containing the .mp4 file
    
    Output 
    --------------------
    result : boolean to check if success
    """

    result = False    
    try:
        base = filename.split(".mp4")[0]
        
        ff = ffmpy3.FFmpeg(
            inputs={src_folder+filename: None},
            outputs={dst_folder+base+'.wav': None}
        )
        ff.run()
        result = True
    except:
        print("something went wrong!!")
    finally:
        return result
    

testing conversion to wav

In [31]:
mp4_to_wav(os.listdir(RAW_DATA_DIR)[0],RAW_DATA_DIR,WAV_DATA_DIR)

True

In [ ]:
def get_sample_rate(f):
    """returns sample rate 
    
    Input : 
    --------------------
    f : wav filename
    
    Output :
    ----------------------
    sr : sample rate of the signal in Hz
    """
    a = AudioSegment.from_wav(f)
    return a.frame_rate


In [45]:
def resample_wav(filename, wav_folder = WAV_DATA_DIR, resampled_folder = WAV_DATA_RESAMPLED_DIR, new_sample_rate =16000):
  """
  resamples the wav file to the given sample rate
  
  Input 
  ------------------
  filename : name of the .wav file
  wav_folder : name of the folder containing the raw wav before resampling
  resampled_folder : destination folder
  
  Output
  ------------------
  result : boolean check if success
  """
  

  result = False
  
  try :

    sound = AudioSegment.from_wav(wav_folder+filename)
    sound = sound.set_frame_rate(new_sample_rate)
    sound.export(resampled_folder+filename, format='wav')
    result = True

  except:
    print("something went wrong!!")
  
  finally:WAV_DATA_RESAMPLED_DIR+os.listdir(WAV_DATA_RESAMPLED_DIR)[0]
    return result

Testing resample function

In [46]:
resample_wav(os.listdir(WAV_DATA_DIR)[0],WAV_DATA_DIR,WAV_DATA_RESAMPLED_DIR,16000)

True

In [76]:
def trim_data(filename,dst_filename,src_folder,dst_folder,start,end):
  """returns a segment from wav file starting "start" to "end" 
  
  Input
  ----------------
  filename : name of the .wav file
  dst_filename: name of the resulting segment
  src_folder : name of the folder containing the wav
  dst_folder : destination folder
  start: beginning time of segment in seconds
  end : end time of segment in seconds

  Output
  ----------------
  result : boolean check if success
  """

  result = False
  
  #checking length related errors

  f = sf.SoundFile(src_folder+filename)
  if f.frames/f.samplerate < end:
    print("out of range")
    return result 

  try:
    start *=1000
    end *=1000

    newAudio = AudioSegment.from_wav(src_folder+filename)
    newAudio = newAudio[start:end]

    newAudio.export(dst_folder+dst_filename, format="wav") #Exports to a wav file in the current path.
    result = True
  except:
    print("something went wrong!!")

  finally:
    return result


testing trim_data

In [81]:
filename = os.listdir(WAV_DATA_RESAMPLED_DIR)[0]
trim_data(filename,"AR_001.wav",WAV_DATA_RESAMPLED_DIR,"./",120,125)

./AR_001.wav


True

In [82]:
import IPython.display as ipd

ipd.Audio("AR_001.wav")